In [1]:
import math
import re
import heapq
import csv

import pandas as pd
import numpy as np

In [2]:
import numpy.linalg as LA

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [26]:
filename = './unique_sorted_split.txt'
df = pd.read_csv(filename, names=["Organization"], quoting=csv.QUOTE_NONE)

In [101]:
outfile = open("text.txt", "w")

threshold = 0.5
vectorizer = CountVectorizer()
transformer = TfidfTransformer()
corpus = set()

for i, s in df.iterrows():
    if not corpus:
        label = s.Organization
        corpus.update(re.sub('[^A-Za-z0-9 ]+', '', s.Organization).split())

    train = vectorizer.fit_transform([' '.join(corpus)]).toarray()
    test = vectorizer.transform([s.Organization]).toarray()
    score = (np.inner(train, test)/(LA.norm(train)*LA.norm(test)))[0][0]

    if np.isnan(score) or score < threshold:
        corpus = set()
        corpus.update(re.sub('[^A-Za-z0-9 ]+', '', s.Organization).split())
        train = vectorizer.fit_transform([' '.join(corpus)]).toarray()
        test = vectorizer.transform([s.Organization]).toarray()
        score = (np.inner(train, test)/(LA.norm(train)*LA.norm(test)))[0][0]
        label = s.Organization

    outfile.write(label + "\n")

outfile.close()

/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:15: RuntimeWarning: invalid value encountered in true_divide
/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:22: RuntimeWarning: invalid value encountered in true_divide
